# Load Cases 🐃 🚚

## 1. Like working with different currencies 💱

Totals (or sub-totals) need to be collected for each load category separately.

In [136]:
DL_2 = 25
LL_2 = 30
SL_2 = 15
WL_2 = 10

DL_1 = 25
LL_1 = 60

DL = DL_1 + DL_2
LL = LL_1 + LL_2
SL = SL_2
WL = WL_2
print(f"{DL=}, {LL=}, {SL=}, {WL=}")

DL=50, LL=90, SL=15, WL=10


## 2. Factoring allows for conversion to "a common currency" 💶

In order to see if there is enough money in the account (member capacity) to pay for the expenses (load demand), we need to convert to a "common currency" by factoring.

In [137]:
FL = 1.25*DL + 1.5*LL + 1.0*SL
print(f"{FL=}")

FL=212.5


## 3. Managing the many possible combinations of factored loads  🏋️‍♀‍💃🌨

### A. Separate functions 🙅

Pros: 
* Once written, each function has its own unique name can be called easily with basic inputs (i.e. just the loads)
* Conceptually simple

Cons:
* Lots of repetitive code to write
* Might be difficult to maintain, easy to introduce bugs through over-repetition
* Function signatures are highly variable (e.g. some funcs only take a `DL` argument, some take `DL`, `LL`, and `SL`). Difficult to write general-purpose functions because many `if-else` statements will need to be written.

In [138]:
def LC1(DL: float) -> float:
    return 1.4 * DL

def LC2(DL: float, LL: float) -> float:
    return 1.25 * DL + 1.5 * LL

def LC2a(DL: float, LL: float, SL: float) -> float:
    return 1.25 * DL + 1.5 * LL + 1.0 * SL

def LC2b(DL: float, LL: float, WL: float) -> float:
    return 0.9 * DL + 1.5 * LL + 0.4 * WL

def LC2c(DL: float, LL: float, WL: float) -> float:
    return 1.25 * DL + 1.5 * LL + 0.4 * WL
    

In [139]:
print(f"{DL=}, {LL=}, {SL=}, {WL=}")
LC2b(DL=DL, LL=LL, WL=WL)

DL=50, LL=90, SL=15, WL=10


184.0

## B. A generic function 👩‍💻

Pros: 
* Write just one function to handle all cases
* Flexible to handle any inputs
* Load combination factors can be stored in an external file: easy to maintain
* Extensible

Cons: 
* Calling the function could get repetitive and tedious, especially if being used in an interactive setting.
* Bad inputs and errors can occur when _calling_ the function in an interactive setting.

In [140]:
def factor_loads(
    DL_factor: float = 0,
    DL: float = 0,
    LL_factor: float = 0,
    LL: float = 0,
    SL_factor: float = 0,
    SL: float = 0,
    WL_factor: float = 0,
    WL: float = 0,
    EL_factor: float = 0,
    EL: float = 0,
) -> float:
    """
    Returns the factored load for the given load factors
    and loads.
    """
    factored_load = (
        DL_factor * DL
        + LL_factor * LL
        + SL_factor * SL
        + WL_factor * WL
        + EL_factor * EL
    )
    return factored_load

In [141]:
lc2 = factor_loads(DL_factor=1.25, DL=DL, LL_factor=1.5, LL=LL)
lc2a = factor_loads(
    DL_factor = 1.25, 
    DL=DL, 
    LL_factor = 1.5, 
    LL=LL, 
    SL_factor=1.0, 
    SL=SL
)
print(f"{DL=}, {LL=}, {SL=}, {WL=}")
print(f"{lc2=}, {lc2a=}")

DL=50, LL=90, SL=15, WL=10
lc2=197.5, lc2a=212.5


## C. Vector math

Pros: 
* Fast
* Flexible for different inputs
* Load combination vectors can be stored in an external file: easy to maintain.

Cons: 
* Arrays are positional but do not have "labelled positions"
* If you need to add an array position, all corresponding arrays need to be updated across all functions (less extensible)

In [142]:
import numpy as np

LC1 = [1.4, 0, 0, 0, 0]
LC2 = [1.25, 1.5, 0, 0, 0]
LC2a = [1.25, 1.5, 1.0, 0, 0]
LC2b = [0.9, 1.5, 0, 0.4, 0]

load = [DL, LL, SL, WL, 0]

def factor_vec_load(load_vector: np.ndarray, factor_vector: np.ndarray) -> float:
    """
    Returns the loads in 'load_vector' factored by 'factor_vector'.
    Both arrays must be the same length.
    """
    return sum(load_vector * factor_vector)
    
fl = factor_vec_load(np.array(load), np.array(LC2a))

print(f"{fl=}")


fl=212.5


## 4. Storing Load Combinations 📚

In [143]:
import json

### Factor Dictionaries 📔

```json
{
    "LC1": {
        "DL_factor": 1.4
    },
    "LC2": {
        "DL_factor": 1.25,
        "LL_factor": 1.5
    },
    "LC2a": {
        "DL_factor": 0.9,
        "LL_factor": 1.5
    },
    "LC2b": {
        "DL_factor": 1.25,
        "LL_factor": 1.5,
        "SL_factor": 1.0
    },
    "LC2c": {
        "DL_factor": 1.25,
        "LL_factor": 1.5,
        "WL_factor": 1.0
    },
    "LC2d": {
        "DL_factor": 0.9,
        "LL_factor": 1.5,
        "WL_factor": 0.4
    }
}
```

In [144]:
with open("NBCC_dict.json", 'r') as json_file:
    nbcc_dict = json.load(json_file)
nbcc_dict

{'LC1': {'DL_factor': 1.4},
 'LC2': {'DL_factor': 1.25, 'LL_factor': 1.5},
 'LC2a': {'DL_factor': 0.9, 'LL_factor': 1.5},
 'LC2b': {'DL_factor': 1.25, 'LL_factor': 1.5, 'SL_factor': 1.0},
 'LC2c': {'DL_factor': 1.25, 'LL_factor': 1.5, 'WL_factor': 1.0},
 'LC2d': {'DL_factor': 0.9, 'LL_factor': 1.5, 'WL_factor': 0.4}}

### Vector dictionaries ➡➡➡➡➡

```json
{
    "LC1": [1.4, 0, 0, 0, 0],
    "LC2": [1.25, 1.5, 0, 0, 0],
    "LC2a": [0.9, 1.5, 0, 0, 0],
    "LC2b": [1.25, 1.5, 1.0, 0, 0],
    "LC2c": [1.25, 1.5, 0, 0.4, 0],
    "LC2d": [0.9, 1.5, 0.4, 0, 0],
}
```

In [145]:
with open('NBCC_vec.json', 'r') as json_file:
    nbcc_vec = json.load(json_file)
nbcc_vec

{'LC1': [1.4, 0, 0, 0, 0],
 'LC2': [1.25, 1.5, 0, 0, 0],
 'LC2a': [0.9, 1.5, 0, 0, 0],
 'LC2b': [1.25, 1.5, 1.0, 0, 0],
 'LC2c': [1.25, 1.5, 0, 0.4, 0],
 'LC2d': [0.9, 1.5, 0.4, 0, 0]}

In [146]:
print(f'{factor_loads(DL=2.4, LL=0.9, SL=3.6, **nbcc_dict["LC2a"])=}')
print(f'{factor_vec_load(np.array([2.4, 0.9, 3.6, 0, 0]), np.array(nbcc_vec["LC2a"]))=}')

factor_loads(DL=2.4, LL=0.9, SL=3.6, **nbcc_dict["LC2a"])=3.5100000000000002
factor_vec_load(np.array([2.4, 0.9, 3.6, 0, 0]), np.array(nbcc_vec["LC2a"]))=3.5100000000000002


## 5. Load Categories 🔢

> Sometimes there are two load "categories" that we may want to treat as being a part of the same load case.
>
> e.g. Both self-weight and super-imposed dead load can be considered "dead load" or `DL`

In [147]:
def transform_to_service_loads(loads: dict, load_categories: dict) -> dict:
    """
    Returns a dictionary of service loads (i.e. loads conforming to the
    NBCC categories of DL, LL, SL, etc.) obtained from the given loads
    in 'loads' correlated against the load types in 'load_categories'.
    
    If the load type in 'loads' is not listed in 'load_categories', no 
    transformation takes place.
    """
    service_loads = {} # acc
    for load_name, load in loads.items():
        load_type = load_categories.get(load_name, load_name)
        if load_type in service_loads:
            service_loads[load_type] = service_loads[load_type] + load
        else:
            service_loads.update({load_type: load})
    return service_loads

### Example

load_categories_lookup = {
    "SW": "DL",
    "SDL": "DL",
    "Planter Soil": "LL",
    "Snow": "SL", 
    "Rain": "SL"
}
loads = {"SW": 14, "SDL": 30, "Planter Soil": 180, "Snow": 30, "Rain": 2}

service_loads = transform_to_service_loads(loads, load_categories_lookup)
print(f"{service_loads=}")

service_loads={'DL': 44, 'LL': 180, 'SL': 32}


## 6. Find maximum factored load

> Two approaches:
> 1. Return just the maximum load
> 2. Return the maximum load AND which load case produced it

### A. Maximum value only (both dict and vector examples)

In [148]:
def max_factored_dict(load: dict, load_combinations: dict) -> float:
    """
    Returns a float representing the maximum factored load of 'load' calculated
    from all of the load combinations in 'load_factors'.
    """
    fl = 0 # acc
    for load_combination, load_factors in load_combinations.items():
        current_fl = factor_loads(**load, **load_factors)
        fl = max(current_fl, fl)
    return fl

### Example
service_loads = {'DL': 44, 'LL': 180, 'SL': 32}
print("Dictionary example: ", f"{service_loads=}")

max_factored = max_factored_dict(service_loads, nbcc_dict)
print(f"{max_factored=}", "\n")


def max_factored_vec(load: list, load_combinations: dict) -> float:
    """
    Returns a float representing the maximum factored load of 'load' calculated
    from all of the load combinations in 'load_factors'.
    """
    fl = 0 # acc
    for load_combination, load_factors in load_combinations.items():
        load_vector = np.array(load)
        factor_vector = np.array(load_factors)
        current_fl = factor_vec_load(load_vector, factor_vector)
        fl = max(current_fl, fl)
    return fl

### Example
service_loads = [44, 180, 32, 0, 0]
print("Vector example: ", f"{service_loads=}")
max_factored = max_factored_vec(service_loads, nbcc_vec)
print(f"{max_factored=}")

Dictionary example:  service_loads={'DL': 44, 'LL': 180, 'SL': 32}
max_factored=357.0 

Vector example:  service_loads=[44, 180, 32, 0, 0]
max_factored=357.0


### B. Maximum value and governing combination (both dict and vector examples)

In [149]:
def max_factored_dict_govern(load: dict, load_combinations: dict) -> float:
    """
    Returns a float representing the maximum factored load of 'load' calculated
    from all of the load combinations in 'load_factors'.
    """
    fl = 0 # acc
    governing_lc = "" # ADDED
    for load_combination, load_factors in load_combinations.items():
        current_fl = factor_loads(**load, **load_factors)
        if current_fl > fl: # ADDED
            governing_lc = load_combination # ADDED
            fl = current_fl # UPDATE
    return fl, governing_lc

### Example

service_loads = {'DL': 44, 'LL': 180, 'SL': 32}
print("Dictionary example: ", f"{service_loads=}")

max_factored = max_factored_dict_govern(service_loads, nbcc_dict)
print(f"{max_factored=}", "\n")
fl, lc = max_factored
print(f"{service_loads=}\n{fl=}\n{nbcc_dict[lc]=}\n\n")


def max_factored_vec_govern(load: list, load_combinations: dict) -> float:
    """
    Returns a float representing the maximum factored load of 'load' calculated
    from all of the load combinations in 'load_factors'.
    """
    fl = 0 # acc
    governing_lc = "" # ADDED
    for load_combination, load_factors in load_combinations.items():
        load_vector = np.array(load)
        factor_vector = np.array(load_factors)
        current_fl = factor_vec_load(load_vector, factor_vector)
        if current_fl > fl: # ADDED
            governing_lc = load_combination # ADDED
            fl = current_fl # UPDATE
    return fl, governing_lc

### Example

service_loads = [44, 180, 32, 0, 0]
print("Vector example: ", f"{service_loads=}")
max_factored = max_factored_vec_govern(service_loads, nbcc_vec)
print(f"{max_factored=}\n")
fl, lc = max_factored
print(f"{service_loads=}\n{fl=}\n{nbcc_vec[lc]=}")

Dictionary example:  service_loads={'DL': 44, 'LL': 180, 'SL': 32}
max_factored=(357.0, 'LC2b') 

service_loads={'DL': 44, 'LL': 180, 'SL': 32}
fl=357.0
nbcc_dict[lc]={'DL_factor': 1.25, 'LL_factor': 1.5, 'SL_factor': 1.0}


Vector example:  service_loads=[44, 180, 32, 0, 0]
max_factored=(357.0, 'LC2b')

service_loads=[44, 180, 32, 0, 0]
fl=357.0
nbcc_vec[lc]=[1.25, 1.5, 1.0, 0, 0]


## 7. Occupancies 🏠 🏢 🏥

> In buildings, loads can often be grouped together in different occupancies.
>
> `dict`s can be useful for organizing these.

In [150]:
import forallpeople as si
si.environment('structural')

### A. Occupancy dictionaries 📔

In [151]:
occupancies_dict = {
    "Residential": {
        "SW": 1.3,
        "SDL": 1.1,
        "LL": 2.4,
    },
    "Residential Roof": {
        "SW": 0.6,
        "SDL": 0.3,
        "Snow": 1.1,
        "Rain": 0.2,
    },
    "Residential Amenity Roof": {
        "SW": 1.4,
        "SDL": 1.0,
        "Planter Soil": 3.6,
        "LL": 4.8,
    },
    "Rooftop Mech": {
         "DL": 1.83,
     },
}

In [152]:
occupancy = "Residential Roof"

service_loads = transform_to_service_loads(
    occupancies_dict[occupancy],
    load_categories_lookup
)

factored = max_factored_dict_govern(
    service_loads,
    nbcc_dict
)

max_fl, governing = factored
print(f"{service_loads=}")
print(f"{max_fl=}, {governing=}")
print(f"{nbcc_dict[governing]}")

service_loads={'DL': 0.8999999999999999, 'SL': 1.3}
max_fl=2.425, governing='LC2b'
{'DL_factor': 1.25, 'LL_factor': 1.5, 'SL_factor': 1.0}


### B. Occupancy vectors ➡➡➡➡➡

> If you are wanting to use vectors for occupancies, it is easiest to use have your vectors already as _service loads_.

In [153]:
# 
occupancies_vec = {
    "Residential": np.array([2.4, 2.4, 0, 0, 0]),
    "Residential Roof": np.array([0.9, 0, 1.3, 0, 0]),
    "Residential Amenity Roof": np.array([2.6, 4.8, 0, 0, 0]),
    "Rooftop Mech": np.array([1.83, 0, 0, 0, 0]),
}

In [154]:
occupancy = "Residential Amenity Roof"
service_loads = occupancies_vec[occupancy]

factored = max_factored_vec_govern(
    service_loads,
    nbcc_vec
)

max_fl, governing = factored
print(f"{service_loads=}")
print(f"{max_fl=}, {governing=}")
print(f"{nbcc_dict[governing]}")

service_loads=array([2.6, 4.8, 0. , 0. , 0. ])
max_fl=10.45, governing='LC2'
{'DL_factor': 1.25, 'LL_factor': 1.5}


# Code Summary 💡

## Functions 🧰

In [155]:
def max_factored_dict_govern(service_load: dict, load_combinations: dict) -> float:
    """
    Returns a float representing the maximum factored load of 'load' calculated
    from all of the load combinations in 'load_factors'.
    """
    fl = 0 # acc
    governing_lc = "" # ADDED
    for load_combination, load_factors in load_combinations.items():
        current_fl = factor_loads(**service_load, **load_factors)
        if current_fl > fl: # ADDED
            governing_lc = load_combination # ADDED
            fl = current_fl # UPDATE
    return fl, governing_lc


def min_factored_dict_govern(service_load: dict, load_combinations: dict) -> float:
    """
    Returns a float representing the minimum factored load of 'load' calculated
    from all of the load combinations in 'load_factors'.
    """
    fl = float('inf') # acc
    governing_lc = "" # ADDED
    for load_combination, load_factors in load_combinations.items():
        current_fl = factor_loads(**service_load, **load_factors)
        if current_fl < fl: # ADDED
            governing_lc = load_combination # ADDED
            fl = current_fl # UPDATE
    return fl, governing_lc


def factor_loads(
    DL_factor: float = 0,
    DL: float = 0,
    LL_factor: float = 0,
    LL: float = 0,
    SL_factor: float = 0,
    SL: float = 0,
    WL_factor: float = 0,
    WL: float = 0,
    EL_factor: float = 0,
    EL: float = 0,
) -> float:
    """
    Returns the factored load for the given load factors
    and loads.
    """
    factored_load = (
        DL_factor * DL
        + LL_factor * LL
        + SL_factor * SL
        + WL_factor * WL
        + EL_factor * EL
    )
    return factored_load


def transform_to_service_loads(loads: dict, load_categories: dict) -> dict:
    """
    Returns a dictionary of service loads (i.e. loads conforming to the
    NBCC categories of DL, LL, SL, etc.) obtained from the given loads
    in 'loads' correlated against the load types in 'load_categories'.
    
    If the load type in 'loads' is not listed in 'load_categories', no 
    transformation takes place.
    """
    service_loads = {} # acc
    for load_name, load in loads.items():
        load_type = load_categories.get(load_name, load_name)
        if load_type in service_loads:
            service_loads[load_type] = service_loads[load_type] + load
        else:
            service_loads.update({load_type: load})
    return service_loads

## Data 💾

In [156]:
# Load Factors from JSON file
with open("NBCC_dict.json", 'r') as json_file:
    nbcc_dict = json.load(json_file)

    
# Project-specific occupancies
occupancies_dict = {
    "Residential": {
        "SW": 1.3,
        "SDL": 1.1,
        "LL": 2.4,
    },
    "Residential Roof": {
        "SW": 0.6,
        "SDL": 0.3,
        "Snow": 1.1,
        "Rain": 0.2,
    },
    "Residential Amenity Roof": {
        "SW": 1.4,
        "SDL": 1.0,
        "Planter Soil": 3.6,
        "LL": 4.8,
    },
    "Rooftop Mech": {
         "DL": 1.83,
     },
}


# Common load categories, specific to engineer
## These can be stored in your own JSON file
load_categories_lookup = {
    "SW": "DL",
    "SDL": "DL",
    "Planter Soil": "LL",
    "Snow": "SL", 
    "Rain": "SL"
}

# Practical Example 🔨

In [157]:
import pycba

span_left = 4.5
udl_left = {
    "SW": 2.4 * span_left,
    "SDL": 20,
    "LL": 100,
}

span_right = 3.2
udl_right = {
    "SW": 2.4 * span_right,
    "SDL": 16,
    "SL": 70,
}

udl_left_service = transform_to_service_loads(udl_left, load_categories_lookup)
udl_right_service = transform_to_service_loads(udl_right, load_categories_lookup)

In [169]:
## Beam
L = [span_left, span_right]
EI = 39e3
R = [-1, 0, -1, 0, 0, 0]

left_reactions = {}
right_reactions = {}
for lc in (udl_left_service | udl_right_service).keys():
    LM = []
    load_span_left = [1, 1, udl_left_service.get(lc, 0), 0, 0]
    load_span_right = [2, 1, udl_right_service.get(lc, 0), 0, 0]
    
    LM = [load_span_left, load_span_right]
    beam = pycba.BeamAnalysis(L, EI, R, LM)
    beam.analyze()

    left_reactions.update({lc: beam.beam_results.R[0]})
    right_reactions.update({lc: beam.beam_results.R[1]})
print(f"{left_reactions=}\n")
print(f"{right_reactions=}")

left_reactions={'DL': 42.35742222222222, 'LL': 225.0, 'SL': -79.64444444444445}

right_reactions={'DL': 172.01857777777775, 'LL': 224.99999999999994, 'SL': 303.6444444444445}


In [168]:
fl_left = min_factored_dict_govern(left_reactions, nbcc_dict)
fl_right = max_factored_dict_govern(right_reactions, nbcc_dict)

print(f"{fl_left=}\n{fl_right=}")

fl_left=(59.300391111111104, 'LC1')
fl_right=(856.1676666666666, 'LC2b')
